# Atividade 01

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [1]:
# Importar bibliotecas necessárias
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from prettytable import PrettyTable

# Definir os caminhos dos arquivos de dados
caminho_features = "./data/UCI HAR Dataset/features.txt"
caminho_labels = "./data/UCI HAR Dataset/activity_labels.txt"
caminho_subtrain = "./data/UCI HAR Dataset/train/subject_train.txt"
caminho_xtrain = "./data/UCI HAR Dataset/train/X_train.txt"
caminho_ytrain = "./data/UCI HAR Dataset/train/y_train.txt"
caminho_subtest = "./data/UCI HAR Dataset/test/subject_test.txt"
caminho_xtest = "./data/UCI HAR Dataset/test/X_test.txt"
caminho_ytest = "./data/UCI HAR Dataset/test/y_test.txt"

# Ler as características e rótulos
caracteristicas = pd.read_csv(caminho_features, header=None, names=['nome_var'], sep=r'\s+')
rotulos = pd.read_csv(caminho_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

# Ler os dados de treinamento
sujeito_treino = pd.read_csv(caminho_subtrain, header=None, names=['subject_id'])
colunas_x_treino = [f"feature_{i}" for i in range(1, len(caracteristicas) + 1)]
X_treino = pd.read_csv(caminho_xtrain, delim_whitespace=True, header=None, names=colunas_x_treino)
y_treino = pd.read_csv(caminho_ytrain, header=None, names=['cod_label'])

# Ler os dados de teste
sujeito_teste = pd.read_csv(caminho_subtest, header=None, names=['subject_id'])
colunas_x_teste = [f"feature_{i}" for i in range(1, len(caracteristicas) + 1)]
X_teste = pd.read_csv(caminho_xtest, delim_whitespace=True, header=None, names=colunas_x_teste)
y_teste = pd.read_csv(caminho_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [2]:
%%time
# Criar e treinar a árvore de decisão com ccp_alpha=0.001
clf = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
clf.fit(X_treino, y_treino.values.ravel())

# Avaliar a acurácia nos conjuntos de treinamento e teste
acc_treino = accuracy_score(y_treino, clf.predict(X_treino))
acc_teste = accuracy_score(y_teste, clf.predict(X_teste))

# Criar uma tabela com os resultados
tabela = PrettyTable(["Métrica", "Valor"])
tabela.add_row(["Acurácia no treinamento", f"{acc_treino:.4f}"])
tabela.add_row(["Acurácia no teste", f"{acc_teste:.4f}"])
print(tabela)

+-------------------------+--------+
|         Métrica         | Valor  |
+-------------------------+--------+
| Acurácia no treinamento | 0.9758 |
|    Acurácia no teste    | 0.8799 |
+-------------------------+--------+
CPU times: total: 3.42 s
Wall time: 4.06 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [3]:
%%time
# Realizar a Análise de Componentes Principais (PCA)
pca = PCA(n_components=1)
X_train_pca = pca.fit_transform(X_treino)
X_test_pca = pca.transform(X_teste)

# Criar e treinar a árvore de decisão com a única componente principal
clf_pca = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
clf_pca.fit(X_train_pca, y_treino.values.ravel())

# Avaliar a acurácia nos conjuntos de treinamento e teste (PCA)
acc_treino_pca = accuracy_score(y_treino, clf_pca.predict(X_train_pca))
acc_teste_pca = accuracy_score(y_teste, clf_pca.predict(X_test_pca))

# Criar uma tabela com os resultados (PCA)
tabela_pca = PrettyTable(["Métrica", "Valor"])
tabela_pca.add_row(["Acurácia no treinamento (PCA)", f"{acc_treino_pca:.4f}"])
tabela_pca.add_row(["Acurácia no teste (PCA)", f"{acc_teste_pca:.4f}"])
print(tabela_pca)

+-------------------------------+--------+
|            Métrica            | Valor  |
+-------------------------------+--------+
| Acurácia no treinamento (PCA) | 0.4997 |
|    Acurácia no teste (PCA)    | 0.4571 |
+-------------------------------+--------+
CPU times: total: 281 ms
Wall time: 215 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [4]:
%%time
# Definir a lista de componentes a serem testados
componentes = [1, 2, 5, 10, 50]

# Criar tabela geral para armazenar os resultados
tabela_geral = PrettyTable(["# Componentes", "Acurácia no Treinamento", "Acurácia no Teste"])

# Para cada número de componentes
for n_comp in componentes:
    print(f"\nUtilizando {n_comp} componente(s) principal(is):")
    
    # Realizar a Análise de Componentes Principais (PCA)
    pca = PCA(n_components=n_comp)
    X_train_pca = pca.fit_transform(X_treino)
    X_test_pca = pca.transform(X_teste)
    
    # Criar um dicionário de parâmetros para GridSearchCV
    param_grid = {
        'max_depth': [3, 5, 7, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    # Criar o GridSearchCV
    grid_search = GridSearchCV(estimator=DecisionTreeClassifier(ccp_alpha=0.001, random_state=42),
                               param_grid=param_grid,
                               scoring='accuracy',
                               cv=5,
                               n_jobs=-1)
    
    # Ajustar o GridSearchCV
    grid_search.fit(X_train_pca, y_treino.values.ravel())
    
    # Avaliar a acurácia nos conjuntos de treinamento e teste com os melhores parâmetros
    clf_pca = grid_search.best_estimator_
    acc_treino_pca = accuracy_score(y_treino, clf_pca.predict(X_train_pca))
    acc_teste_pca = accuracy_score(y_teste, clf_pca.predict(X_test_pca))
    
    # Adicionar resultados à tabela geral
    tabela_geral.add_row([n_comp, f"{acc_treino_pca:.4f}", f"{acc_teste_pca:.4f}"])

print(tabela_geral)


Utilizando 1 componente(s) principal(is):

Utilizando 2 componente(s) principal(is):

Utilizando 5 componente(s) principal(is):

Utilizando 10 componente(s) principal(is):

Utilizando 50 componente(s) principal(is):
+---------------+-------------------------+-------------------+
| # Componentes | Acurácia no Treinamento | Acurácia no Teste |
+---------------+-------------------------+-------------------+
|       1       |          0.4954         |       0.4693      |
|       2       |          0.6128         |       0.5847      |
|       5       |          0.8460         |       0.7886      |
|       10      |          0.8927         |       0.8246      |
|       50      |          0.9029         |       0.8174      |
+---------------+-------------------------+-------------------+
CPU times: total: 3.31 s
Wall time: 17.8 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

---------------------------------------------------------------

A árvore de decisão, sem a aplicação do PCA, apresentou uma acurácia superior, porém exigiu um tempo de processamento maior;

O aumento no número de componentes principais melhorou consideravelmente a acurácia do modelo de Árvore de Decisão, tanto no conjunto de treinamento quanto no conjunto de teste;

À medida que mais componentes são adicionados, o tempo de processamento tende a aumentar, pois o modelo precisa lidar com um maior número de variáveis;

Existe um trade-off entre acurácia e velocidade de processamento.